# 실습 세션 3: 여행 플래너 Multi-Agent 앱의 Docker 빌드 및 AKS 배포

이 세션에서는 세션 2에서 만든 여행 플래너 멀티에이전트 앱을 Docker 이미지로 빌드하고, Azure Kubernetes Service(AKS)에 배포하는 핵심 실습만 다룹니다.

## 1. Docker Desktop 설치

Docker 이미지를 빌드하고 테스트하려면 로컬에 Docker Desktop이 설치되어 있어야 합니다.

- [Docker Desktop 다운로드(Windows/Mac)](https://www.docker.com/products/docker-desktop/)
- 설치 후, 터미널에서 `docker --version` 명령어로 정상 설치 여부를 확인하세요.

설치가 완료되면 Docker Desktop을 실행한 상태에서 아래 실습을 진행하세요.

## 2. Dockerfile 작성

아래는 여행 플래너 앱을 컨테이너화하기 위한 Dockerfile 예시입니다.

In [ ]:
%%writefile Dockerfile
# Step 1: Base image with Python environment
FROM python:3.12-slim AS base

# Step 2: Set working directory
WORKDIR /app

# Step 3: Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Step 4: Copy the project files into the container
COPY . .

# Step 5: Define the entry point
ENTRYPOINT ["python", "/app/travel_planning_agent.py"]

## 2. Docker 이미지 빌드

아래 명령어로 Docker 이미지를 빌드합니다. (로컬에서 실행)

In [ ]:
# Docker 이미지 빌드
!docker build -t travel-multiagent-app:v1 .

## 3. AKS 클러스터 및 ACR 레지스트리 생성

아래 명령어로 Azure에서 컨테이너 레지스트리(ACR)와 AKS 클러스터을 생성합니다. (Azure CLI 필요)

In [ ]:
# Azure 리소스 그룹 생성 (예시)
# !az group create --name myResourceGroup --location koreacentral

# Azure Container Registry 생성 (예시)
# !az acr create --resource-group myResourceGroup --name <ACR-REGISTRY-NAME> --sku Basic

# AKS 클러스터 생성 (예시)
# !az aks create --resource-group myResourceGroup --name myAKSCluster --node-count 1 --enable-addons monitoring --generate-ssh-keys --attach-acr <ACR-REGISTRY-NAME>

# AKS 클러스터 인증 정보 가져오기
# !az aks get-credentials --resource-group myResourceGroup --name myAKSCluster

이제 위에서 생성한 ACR과 AKS를 활용해 이미지를 푸시하고, 클러스터에 배포할 수 있습니다.

## 4. Kubernetes 배포를 위한 YAML 파일 작성

아래는 AKS에 배포할 때 사용할 수 있는 예시 deployment.yaml 및 service.yaml 입니다.

In [ ]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: travel-multiagent-app
spec:
  replicas: 1
  selector:
    matchLabels:
      app: travel-multiagent-app
  template:
    metadata:
      labels:
        app: travel-multiagent-app
    spec:
      containers:
      - name: travel-multiagent-app
        image: <ACR-REGISTRY>/travel-multiagent-app:v1  # 실제 ACR 경로로 교체

## 5. AKS에 배포 (kubectl)

아래 명령어로 이미지를 ACR에 푸시하고, AKS에 배포합니다.

In [ ]:
# 1. ACR에 이미지 푸시 (예시)
# !az acr login --name <ACR-REGISTRY-NAME>
# !docker tag travel-multiagent-app:v1 <ACR-REGISTRY>/travel-multiagent-app:v1
# !docker push <ACR-REGISTRY>/travel-multiagent-app:v1

# 2. AKS에 배포
# !kubectl apply -f deployment.yaml

---

- `<ACR-REGISTRY>`는 실제 Azure Container Registry 주소로 교체하세요.
- 환경 변수/시크릿은 travel-multiagent-env Kubernetes Secret으로 관리할 수 있습니다.
- 필요시 replicas, 리소스, 네트워크 설정 등은 YAML에서 조정하세요.

## 6. Log Analytics Workspace에서 컨테이너 로그 조회

AKS 클러스터를 생성할 때 Log Analytics Workspace를 연동했다면, 아래 방법으로 컨테이너 로그를 확인할 수 있습니다.

- Azure Portal에서 AKS 클러스터 > 모니터링 > 로그(Logs) 메뉴로 이동
- 아래와 같은 Kusto 쿼리로 컨테이너 로그를 조회할 수 있습니다.

In [ ]:
# 예시: 컨테이너 v2 로그 Kusto 쿼리
# ContainerLogV2
# | where ContainerName == "travel-multiagent-app"
# | sort by TimeGenerated desc
# | take 100

- 필요에 따라 PodName, Namespace, LogLevel 등으로 필터링할 수 있습니다.
- 실시간 로그 스트림이 필요하다면 Azure Portal의 "실시간 로그" 기능을 활용하거나, kubectl logs 명령어도 사용할 수 있습니다.